# Connect to Reddit API

In [1]:
import praw

reddit = praw.Reddit(
    client_id="0Zpkb9lo65TvAHdWgOvszQ",
    client_secret="EhEqZ9F5l84VYg2HzXtLYY-DoXNvHg",
    user_agent="data-extraction-script",
)

# Specialized subreddits (EV/Petrol Cars)
## Create subreddit object

## Related subreddits
- Petrol cars
r/cars/
- Electric cars
r/electricvehicles/
r/nzev
r/electriccars/

# Post docs: https://praw.readthedocs.io/en/stable/code_overview/models/submission.html#submission
# Comments docs: https://praw.readthedocs.io/en/stable/code_overview/models/comment.html

In [2]:
import csv
import emoji
import time
from praw.models import MoreComments

def write_headers(subreddit_name):
    # Write header first
    with open(f'{subreddit_name}-posts.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            'author',
            'author_flair_text',
            'clicked',
            'created_utc',
            'distinguished',
            'edited',
            'id',
            'is_original_content',
            'is_self',
            'link_flair_text',
            'locked',
            'name',
            'num_comments',
            'over_18',
            'permalink',
            'saved',
            'score',
            'selftext',
            'spoiler',
            'stickied',
            'title',
            'upvote_ratio',
            'url'
        ])
        
    with open(f'{subreddit_name}-comments.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([
            'body',
            'body_html',
            'created_utc',
            'distinguished',
            'edited',
            'id',
            'is_submitter',
            'link_id',
            'parent_id',
            'permalink',
            'saved',
            'score',
            'stickied',
            'subreddit_id'
        ])

# Errors Documentation
- instances of the MoreComments class, which represent "load more comments" links on the Reddit website.

In [3]:
def extract_all_post_comments_post_ids(posts, subreddit_name, post_ids=[]):
    # Prep headers first
    write_headers(subreddit_name)
    
    for index, post in enumerate(posts, start=1):
        # Only write unique posts & comments
        if post.id not in post_ids:
            post_row = [
                getattr(post, 'author', '[deleted]') or '[deleted]', # Some accounts are deleted
                getattr(post, 'author_flair_text', 'None') or 'None',
                getattr(post, 'clicked', None) or 'None',
                # post.comments, # No need instance
                getattr(post, 'created_utc', None) or 'None',
                getattr(post, 'distinguished', None) or 'None',
                getattr(post, 'edited', None) or 'None',
                getattr(post, 'id', None) or 'None',
                getattr(post, 'is_original_content', None) or 'None',
                getattr(post, 'is_self', None) or 'None',
                #post.link_flair_template_id,
                getattr(post, 'link_flair_text', None) or 'None',
                getattr(post, 'locked', None) or 'None',
                getattr(post, 'name', None) or 'None',
                getattr(post, 'num_comments', None) or 'None',
                getattr(post, 'over_18', None) or 'None',
                getattr(post, 'permalink', None) or 'None',
                # post.poll_data,
                getattr(post, 'saved', None) or 'None',
                getattr(post, 'score', None) or 'None',
                getattr(post, 'selftext', None) or 'None',
                getattr(post, 'spoiler', None) or 'None',
                getattr(post, 'stickied', None) or 'None',
                # post.subreddit, # No need instance
                getattr(post, 'title', None) or 'None',
                getattr(post, 'upvote_ratio', None) or 'None',
                getattr(post, 'url', None) or 'None'
            ]
            print(f"No: {index}, Title: {subreddit_name}:{post.title}, PostID: {post.id}")
            
            with open(f'{subreddit_name}-posts.csv', 'a', newline='', encoding='utf-8') as f:
                # Write to file
                writer = csv.writer(f)
                writer.writerow(post_row)
                
            time.sleep(60) # Sleep for 60 seconds to avoid rate limit
    
            # Write post comments too
            post = reddit.submission(id=post_row[6]) # post.id
            
            # post.comments.replace_more(limit=None)  # Fetch all comments replies
            # for comment in post.comments.list():
            #     # Skip comments that are deleted
            #     if comment.body == '[deleted]':
            #         continue
            # If you want to fetch all comments, comment out skipping of MoreComments
            
            for comment in post.comments:
                if isinstance(comment, MoreComments):
                    # Skip lower level comments
                    continue
                # Skip comments that are deleted
                if comment.body == '[deleted]' or comment.body == '[removed]':
                    continue

                comment_row = [
                    # comment.author, # No need instance
                    emoji.demojize(getattr(comment, 'body', 'None')),
                    emoji.demojize(getattr(comment, 'body_html', 'None')),
                    getattr(comment, 'created_utc', None) or 'None',
                    getattr(comment, 'distinguished', None) or 'None',
                    getattr(comment, 'edited', None) or 'None',
                    getattr(comment, 'id', None) or 'None',
                    getattr(comment, 'is_submitter', None) or 'None',
                    getattr(comment, 'link_id', None) or 'None',
                    getattr(comment, 'parent_id', None) or 'None',
                    getattr(comment, 'permalink', None) or 'None',
                    # comment.replies, # No need instance
                    getattr(comment, 'saved', None) or 'None',
                    getattr(comment, 'score', None), # Downvotes - Upvotes
                    getattr(comment, 'stickied', None) or 'None',
                    # comment.submission, # No need instance
                    # comment.subreddit, # No need instance
                    getattr(comment, 'subreddit_id', None) or None
                ]
    
                with open(f'{subreddit_name}-comments.csv', 'a', newline='', encoding='utf-8') as f:
                    writer = csv.writer(f)
                    writer.writerow(comment_row)
                    
            post_ids.append(post.id)
    return post_ids

# Extract specialized subreddits

In [4]:
specialized_subreddits = [
    # "electricvehicles",
    # "electriccars",
    # "nzev",
    # "ElectricCarUK",
    # "teslamotors",
    # "RealTesla",
    # "evcharging",
    # "TeslaModel3",
]

for subreddit_name in specialized_subreddits:
    subreddit = reddit.subreddit(subreddit_name)
    top_posts = subreddit.top(limit=100)
    extract_all_post_comments_post_ids(top_posts, subreddit_name)

No: 1, Title: nzev:This is neat: an electric vehicle towing an electric excavator, both grabbing a rapid charge., PostID: 1b1m8f5
No: 2, Title: nzev:Enough about RUCs, where's my EV camping crew at?, PostID: 1ak27c6
No: 3, Title: nzev:Enough is enough, PostID: 1al8gv0
No: 4, Title: nzev:NZ Bloke post a TikTok vid bashing EV cars., PostID: 18bx2tq
No: 5, Title: nzev:Admin fees are worse than ticketek, PostID: 19eb37j
No: 6, Title: nzev:Seems ute owners pretty excited about having new chargers at Bombay /s, PostID: 194w1vl
No: 7, Title: nzev:What will they do when they are no longer free 😬, PostID: 1ar044v
No: 8, Title: nzev:Just stopped at the Bombay Hills chargers to find the new ones are already installed and active, meaning there is now charging for 12 electric cars here. Also, Z Energy across the road have installed two new chargers as well !, PostID: 18ksgru
No: 9, Title: nzev:Kind of sums up Stuff comments section on any ev's...., PostID: 13dbjg8
No: 10, Title: nzev:"Ute tax" lie 

# Extract general subreddits with search filters

In [5]:
general_subreddits = [
    "cars",
]

search_terms = [
    'petrol',
    'diesel',
    'gasoline',
    'electric',
]

for subreddit_name in general_subreddits:
    post_ids = []
    for search_term in search_terms:
        subreddit = reddit.subreddit(subreddit_name)
        search_results = subreddit.search(search_term, sort='relevance', limit=100)
        post_ids.extend(extract_all_post_comments_post_ids(search_results, f'{subreddit_name}-{search_term}', post_ids))
        post_ids = list(set(post_ids))

No: 1, Title: cars-petrol:New peak rates mean charging electric cars more expensive than petrol, PostID: 10l3ux1
No: 2, Title: cars-petrol:Porsche investigates alleged manipulation of petrol engines, PostID: ifoqxm
No: 3, Title: cars-petrol:UK delays petrol and diesel sales ban, PostID: 16oegun
No: 4, Title: cars-petrol:Ireland: New laws to ban sale of petrol and diesel cars by 2030, PostID: ehma66
No: 5, Title: cars-petrol:Hyundai’s Petrol Hot Hatches Are Officially Dead In Europe, PostID: 1ay4onu
No: 6, Title: cars-petrol:Lotus abandons petrol power for all-electric future - The next Lotus sports car will be petrol, but after that all of the brand’s future cars will be full EVs, PostID: h9lob8
No: 7, Title: cars-petrol:Australian fuel efficiency standards aimed at cutting petrol bills, 'catching up' with US, PostID: 1aj64ej
No: 8, Title: cars-petrol:Petrol Heads who live in apartment buildings with limited space… where do you work on your car?, PostID: 1448aid
No: 9, Title: cars-petr

# Get dataset token summaries

In [5]:
import os
import csv

dataset_dir = 'data'

# Loop through all files in the directory
for file in os.listdir(dataset_dir):
    if file.endswith(".csv"):
        with open(f'{dataset_dir}/{file}', 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            data = list(reader)
            print(f"File: {file}, Rows: {len(data)}")

File: cars-diesel-comments.csv, Rows: 2579
File: cars-diesel-posts.csv, Rows: 95
File: cars-electric-comments.csv, Rows: 5580
File: cars-electric-posts.csv, Rows: 99
File: cars-gasoline-comments.csv, Rows: 2327
File: cars-gasoline-posts.csv, Rows: 100
File: cars-petrol-comments.csv, Rows: 2212
File: cars-petrol-posts.csv, Rows: 101
File: electriccars-comments.csv, Rows: 2287
File: electriccars-posts.csv, Rows: 101
File: ElectricCarUK-comments.csv, Rows: 100
File: ElectricCarUK-posts.csv, Rows: 101
File: electricvehicles-comments.csv, Rows: 6922
File: electricvehicles-posts.csv, Rows: 101
File: evcharging-comments.csv, Rows: 1435
File: evcharging-posts.csv, Rows: 101
File: nzev-comments.csv, Rows: 1579
File: nzev-posts.csv, Rows: 101
File: RealTesla-comments.csv, Rows: 10811
File: RealTesla-posts.csv, Rows: 101
File: TeslaModel3-comments.csv, Rows: 16389
File: TeslaModel3-posts.csv, Rows: 101
File: teslamotors-comments.csv, Rows: 11864
File: teslamotors-posts.csv, Rows: 101
